In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pandas as pd
import re
import selenium

In [2]:
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": ChromeDriverManager().install()}
    return Browser("chrome", **executable_path, headless=False)

In [4]:
browser = init_browser()
url = "https://www.akc.org/dog-breeds/"
browser.visit(url)

#THERE ARE 24 PAGES OF DOG BREEDS
#https://www.akc.org/dog-breeds/page/24/

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280


[WDM] - Driver [/Users/jenniferdean/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


In [7]:
breeds_list_of_dicts = []

for p in range(24):
    p += 1
    #####GO TO NEW BREED PAGE#####
    url = f"https://www.akc.org/dog-breeds/page/{p}/"
    browser.visit(url)
    
    for i in range(12):
        html = browser.html
        soup = bs(html, "html.parser")
        
        #####GO TO BREED DETAIL PAGE#####
        browser.find_by_css('a[class="d-block relative"]')[i].click()
        time.sleep(1)
        
        html = browser.html
        soup = bs(html, "html.parser")

        dog_breed = soup.find("h1").get_text()
        dog_breed = dog_breed.replace('\n', '').strip(" ")

        attribute_block = soup.find_all("span", class_="attribute-list__description")
        
        if len(attribute_block) > 5
            temperment = soup.find("span", class_="attribute-list__description attribute-list__text attribute-list__text--lg mb4 bpm-mb5 pb0 d-block").get_text()
            akc_rank = soup.find_all("span", class_="attribute-list__description attribute-list__text")[0].get_text()
            height = soup.find_all("span", class_="attribute-list__description attribute-list__text")[1].get_text()
            weight = soup.find_all("span", class_="attribute-list__description attribute-list__text")[2].get_text()
            life_ex = soup.find_all("span", class_="attribute-list__description attribute-list__text")[3].get_text()
            group = soup.find_all("span", class_="attribute-list__description attribute-list__text")[4].get_text()
        else: 
            temperment = soup.find("span", class_="attribute-list__description attribute-list__text attribute-list__text--lg mb4 bpm-mb5 pb0 d-block").get_text()
            height = soup.find_all("span", class_="attribute-list__description attribute-list__text")[0].get_text()
            weight = soup.find_all("span", class_="attribute-list__description attribute-list__text")[1].get_text()
            life_ex = soup.find_all("span", class_="attribute-list__description attribute-list__text")[2].get_text()
            group = soup.find_all("span", class_="attribute-list__description attribute-list__text")[3].get_text()
        
        image_url = soup.find_all('img')[4]["data-src"]
        
        description = soup.find("div", class_="breed-hero__footer").get_text()
        description = description.replace('\n', '').strip(" ")
        
        brushing = soup.find_all("div", class_="bar-graph__section")[0]["style"].strip('width: ;')
        shedding = soup.find_all("div", class_="bar-graph__section")[1]["style"].strip('width: ;')
        energy = soup.find_all("div", class_="bar-graph__section")[2]["style"].strip('width: ;')
        trainability = soup.find_all("div", class_="bar-graph__section")[3]["style"].strip('width: ;')
        temperment_scale = soup.find_all("div", class_="bar-graph__section")[4]["style"].strip('width: ;')
        
        #####COLOR TABLE#####
        table = soup.find_all('table')[0]
        
        color_list = []
        final_color_list = []

        for row in table.find_all('td'):
            if 'Description' in row:
                continue
            elif 'Standard Colors' in row:
                continue
            elif 'Registration Code' in row:
                continue
            elif "Check Mark For Standard Color" in row:
                continue
            elif "\n" in row:
                continue
            else:
                color = row.get_text()
                color_list.append(color)
            for i in color_list:
                if "Check Mark For Standard Color" in i:
                    continue
                if "\n" in i:
                    continue
                else:
                    try:
                        if int(i) > 0:
                            continue
                    except:
                        final_color_list.append(i)
        
        #####GO BACK TO BREED PAGE#####
        browser.back()
        
        #####APPEND TO THE DICTIONARY#####
        breeds_list_of_dicts.append({"breed_name" : dog_breed,
                         "temperment": temperment,
                         "image" : image_url,
                         "description" : description,
                         "akc_rank" : akc_rank,
                         "height" : height,
                         "weight" : weight,
                         "life_expectancy" : life_ex,
                         "brushing_scale" : brushing,
                         "shedding_scale" : shedding,
                         "energy_scale" : energy,
                         "trainability_scale" : trainability,
                         "temperment_scale" : temperment_scale,
                         "color_options" : final_color_list
                        })
        time.sleep(1)
        
browser.quit()

IndexError: list index out of range

In [8]:
breeds_list_of_dicts

[{'breed_name': 'American Water Spaniel',
  'temperment': 'Eager, Happy, Charming',
  'image': 'https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2017/11/28201602/American-Water-Spaniel-standing-outdoors.jpg',
  'description': 'The American Water Spaniel, native to the lake country of the upper Midwest, is an upbeat, outdoorsy athlete who loves to hunt and swim. They are known to be happy, eager, and charming, though aloof with strangers and a bit stubborn.',
  'akc_rank': 'Ranks 166 of 197',
  'height': '15-18 inches',
  'weight': '30-45 pounds (male), 25-40 pounds (female)',
  'life_expectancy': '10-14 years',
  'brushing_scale': '60%',
  'shedding_scale': '20%',
  'energy_scale': '60%',
  'trainability_scale': '100%',
  'temperment_scale': '80%',
  'color_options': ['Brown',
   'Brown',
   'Brown',
   'Chocolate',
   'Brown',
   'Chocolate',
   'Brown',
   'Chocolate',
   'Liver',
   'Brown',
   'Chocolate',
   'Liver']},
 {'breed_name': 'Anatolian Shepherd Dog',
  

In [ ]:
#American Bulldog - NOT in AKC
#Appenzeller Sennenhund

In [12]:
len(soup.find_all("span", class_="attribute-list__description")) #Appenzeller Sennenhund

5

In [21]:
len(soup.find_all("span", class_="attribute-list__description")) #Affenpinscher

6

In [30]:
soup.find_all('img')[4]["data-src"] #American Bulldog

'https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2019/12/31141719/American-Bulldog.6.jpg'

In [24]:
html = browser.html
soup = bs(html, "html.parser")

In [23]:
len(soup.find_all("span", class_="attribute-list__description"))

6